In [10]:
import sys
sys.path.append('../')  

from data.dummy_datasets import create_cube
from utils.operations import E2G
from models.gnca import GNCAModel
from utils.plots import *
from utils.operations import *
import torch
import torch.nn.functional as F
import torch.optim as optim
import networkx as nx

In [11]:
coords, E = create_cube(3, 1)
G = E2G(E, 27)
adj = G.adjacency()
x = np.zeros(coords.shape[0])
x[coords.shape[0] // 2] = 1
target = np.ones(coords.shape[0])

Training

In [12]:
# Initialize the GNCA model
gnca = GNCAModel(input_dim=16, channel_n=16, fire_rate=0.5)
optimizerG = optim.Adam(gnca.parameters(), lr=2e-2)
epochs = 4000

def loss_f(yhat, y):
    return F.mse_loss(yhat, y)

# Training loop
for epoch in range(epochs):
    
    gnca.train()
    
    x = gnca(x, adj)  

    loss = torch.mean(loss_f(x, target))

    optimizerG.zero_grad()
    loss.backward()

    for param in gnca.parameters():
        if param.grad is not None:
            param.grad.data /= (torch.norm(param.grad.data) + 1e-8)

    optimizerG.step()

    # Print progress
    if epoch % 10 == 0:
        print(f'Step: {epoch}, Loss: {loss.item():.4f}')

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed